In [2]:
import os
import json
from transformers import pipeline

# Désactiver CUDA si le pilote est trop ancien
os.environ["CUDA_VISIBLE_DEVICES"] = ""  # Forcer l'utilisation du CPU

# Charger les données clusterisées
def load_clustered_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Charger un modèle de génération de résumé (T5)
def load_summarizer():
    try:
        return pipeline("summarization", model="t5-small")  # Utiliser le CPU
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {e}")
        return None

# Fonction pour générer un résumé avec T5
def generate_summary_t5(text, max_length=130, min_length=30):
    try:
        summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Erreur lors de la génération du résumé : {e}")
        return "Résumé non disponible"

# Chemin vers le fichier clusterisé
clustered_file_path = 'clustered_articles2.json'
clustered_data = load_clustered_data(clustered_file_path)

# Regrouper les articles par cluster
clusters = {}
for article in clustered_data:
    cluster_id = article['cluster']
    if cluster_id not in clusters:
        clusters[cluster_id] = []
    clusters[cluster_id].append(article['content'])

# Charger le modèle T5
summarizer = load_summarizer()
if summarizer is None:
    print("Impossible de charger le modèle. Vérifiez les erreurs ci-dessus.")
    exit(1)

# Générer un résumé pour chaque cluster
cluster_summaries = {}
for cluster_id, contents in clusters.items():
    # Concaténer le contenu de tous les articles du cluster
    combined_content = " ".join(contents)
    
    # Limiter la longueur du texte pour respecter les limites du modèle
    max_input_length = 512  # Limite de tokens pour T5
    if len(combined_content) > max_input_length:
        combined_content = combined_content[:max_input_length]
    
    # Générer un résumé avec T5
    summary = generate_summary_t5(combined_content, max_length=130, min_length=30)
    cluster_summaries[cluster_id] = summary

# Sauvegarder les résumés dans un fichier JSON
output_file_path = 'cluster_summaries_t5.json'
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(cluster_summaries, file, ensure_ascii=False, indent=4)

print(f"Les résumés ont été enregistrés dans '{output_file_path}'.")

Device set to use cpu


Les résumés ont été enregistrés dans 'cluster_summaries_t5.json'.
